In [2]:
%matplotlib inline

import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy.stats import randint, uniform
from sklearn import linear_model, preprocessing
from sklearn.decomposition import PCA
from sklearn.ensemble import (AdaBoostClassifier, BaggingClassifier,
                              RandomForestClassifier, RandomForestRegressor)
from sklearn.linear_model import Lasso, LogisticRegression, Ridge
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, mean_absolute_error, r2_score)
from sklearn.model_selection import (GridSearchCV, RandomizedSearchCV,
                                     train_test_split)
from sklearn.preprocessing import (LabelEncoder, MinMaxScaler, OneHotEncoder,
                                   PolynomialFeatures, RobustScaler,
                                   StandardScaler)
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from skopt import BayesSearchCV

pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)

In [3]:
# import data
dataset_og = pd.read_csv('Data\Measurements-Transformed')
# kopie maken indien we iets van de originele data nodig hebben
dataset = dataset_og.copy()
dataset.head(5)

,ID,Sex,Measurement_Age,Add,Sph-Far-R,Cyl-Far-R,Axis-Far-R,Sph-Close-R,Cyl-Close-R,Axis-Close-R,Sph-Far-L,Cyl-Far-L,Axis-Far-L,Sph-Close-L,Cyl-Close-L,Axis-Close-L
0,203795.0,0.0,21118.0,0.0,-1.75,0.50,55.0,-2.25,1.0,55.0,-1.75,1.00,110.0,-1.25,0.5,110.0
1,203795.0,0.0,20245.0,0.0,-1.75,0.50,65.0,0.00,0.0,0.0,-1.25,0.50,110.0,0.00,0.0,0.0
2,203795.0,0.0,18099.0,0.0,-1.50,0.50,65.0,0.00,0.0,0.0,-1.00,0.50,110.0,0.00,0.0,0.0
3,546632.0,1.0,13825.0,0.0,-3.50,1.50,180.0,-3.50,1.5,180.0,-3.00,1.50,180.0,-3.00,1.5,180.0
4,546632.0,1.0,9653.0,0.0,-2.00,0.75,175.0,0.00,0.0,0.0,-2.00,0.75,180.0,0.00,0.0,0.0


In [4]:
#drop rijen waar < n meting van zijn en houd van de overige de top n meest recente waardes
dataset = dataset.groupby('ID').filter(lambda x: len(x) > 2)
dataset = dataset.groupby('ID').head(3)

Rijen aan elkaar zetten

In [6]:
#3 rijen

from functools import reduce
dfs = [dataset, dataset, dataset]
dataset = reduce(lambda left, right: pd.merge(left, right, on=['ID', 'Sex']), dfs)

dataset = dataset.sort_values(by=['ID', 'Measurement_Age_x'])
dataset = dataset.groupby('ID').head(4)
dataset = dataset.drop_duplicates(subset=['ID', 'Sex'], keep='last')


In [15]:
# Splitsen in features en targets

y = dataset['Sph-Far-R'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

In [16]:
# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [17]:
# Linear regression

regmodel = linear_model.LinearRegression()
regmodel.fit(X_train,y_train)
print(regmodel.coef_)
r2 = regmodel.score(X_test,y_test)
print('r2 score = ', r2)

[-7.83576650e-08  1.46822058e-02  1.26980281e-05 -1.44787496e-01
  3.23005634e-01  1.86030099e-01 -7.14856203e-04  1.96803415e-02
 -1.26848257e-02  9.50275871e-04 -1.13955369e-02 -1.17471601e-01
 -1.19744849e-03 -1.21558321e-02  5.88739512e-02  6.85877036e-04
  4.16342968e-05 -1.37505102e-01  5.35501323e-01  3.85533963e-03
 -1.28585954e-03  2.27430301e-02 -2.98996123e-01  1.34219285e-03
  1.16703049e-01  1.39922799e-01 -4.49661214e-04  7.48144024e-02
 -7.45530569e-03  1.03565911e-03  5.61001159e-06]
r2 score =  0.8173311267041556


In [27]:
# Splitsen in features en targets

y = dataset['Cyl-Far-R'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Linear regression

regmodel = linear_model.LinearRegression()
regmodel.fit(X_train,y_train)
print(regmodel.coef_)
r2 = regmodel.score(X_test,y_test)
print('r2 score = ', r2)

[-2.79359625e-09  6.73360140e-03 -1.13734596e-05 -7.09079555e-03
 -1.58078220e-02  2.62997797e-01 -3.87579836e-04  3.80026519e-03
  9.53204606e-02  6.19596221e-04  8.39908120e-03  6.06826914e-03
  4.56771632e-05  5.84009989e-03 -8.48228825e-02 -3.58971586e-04
 -1.80995266e-05 -6.09997320e-02  1.35765819e-02  4.65932497e-01
 -3.41612520e-04 -1.33597315e-02  1.43154006e-01  3.30589570e-04
 -8.53352533e-03  1.34878183e-01 -9.17432539e-05  1.12860143e-02
  9.21524914e-02  3.73590465e-04  3.32836264e-05]
r2 score =  0.5680110150424938


In [19]:
# Splitsen in features en targets

y = dataset['Sph-Far-L'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Linear regression

regmodel = linear_model.LinearRegression()
regmodel.fit(X_train,y_train)
print(regmodel.coef_)
r2 = regmodel.score(X_test,y_test)
print('r2 score = ', r2)

[-9.01490900e-08  1.33840191e-02  2.37792260e-05 -1.33426338e-01
  3.18383727e-02  2.47639218e-01 -2.35501050e-03 -4.01756031e-02
  2.28788770e-01  1.11155831e-03  2.59889075e-01 -2.56682155e-01
  2.75141056e-04  1.91454751e-02 -2.60204166e-01  9.49902484e-04
  5.13173384e-05 -1.50533361e-01  1.01068799e-01 -1.81748509e-01
 -6.38431637e-04 -2.45774513e-03 -3.66282242e-01  2.09302147e-03
  5.83125896e-01  4.02005465e-01 -8.85637610e-04  1.01577026e-01
  2.11819643e-02  6.19122924e-04 -1.32256464e-05]
r2 score =  0.7278520879850259


In [20]:
# Splitsen in features en targets

y = dataset['Cyl-Far-L'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Linear regression

regmodel = linear_model.LinearRegression()
regmodel.fit(X_train,y_train)
print(regmodel.coef_)
r2 = regmodel.score(X_test,y_test)
print('r2 score = ', r2)

[-3.38363816e-09  1.57234842e-02 -1.05118504e-05 -5.34970252e-03
 -3.13882076e-03 -1.47373092e-02 -1.73031447e-05  1.72895947e-02
 -9.52700296e-02  1.03989042e-04 -2.94650068e-03  2.82047440e-01
 -2.97916434e-04 -8.79890371e-03  8.86804812e-02  3.51694150e-04
 -1.92690376e-05 -8.00230153e-02 -1.10100775e-02  1.28174544e-01
 -3.60416563e-05  6.69850386e-04  1.58022844e-01  1.26890662e-04
  1.34861419e-02  4.93047498e-01 -4.18477264e-04  8.33232970e-03
  1.09967938e-01  3.15063517e-04  3.40941606e-05]
r2 score =  0.6201286424508907


In [24]:
# Modeloptimalisatie en Hyperparameter tuning


# Automatisch toevoegen van hogere orde features

from sklearn.preprocessing import PolynomialFeatures


# Aanmaken van de hogere orde features
graad = 2

poly = PolynomialFeatures(graad)
poly.fit(X_train)
X_train_poly = poly.transform(X_train)
X_test_poly = poly.transform(X_test)
print('dimensie van X_train_poly: ',X_train_poly.shape)
print('dimensie van X_test_poly: ',X_test_poly.shape)


# met L2 regularisatie via Ridge regression
lregmodel_poly = Ridge(alpha=20,tol=0.0001,fit_intercept=True)
lregmodel_poly.fit(X_train_poly,y_train)

print('R2 score op test set via L2: ',lregmodel_poly.score(X_test_poly,y_test))
# R2 -score via L2 op de trainingset
print('R2 score op training set via L2: ',lregmodel_poly.score(X_train_poly,y_train))


# met L1 regularisatie via Lasso regression
lregmodel_poly = Lasso(alpha=3,tol=0.0001,fit_intercept=True)
lregmodel_poly.fit(X_train_poly,y_train)      
  
    
print('R2 score op test set via L1: ',lregmodel_poly.score(X_test_poly,y_test))
  
# R2 -score via L1 op de trainingset
print('R2 score op training set via L1: ',lregmodel_poly.score(X_train_poly,y_train))  

dimensie van X_train_poly:  (5376, 528)
dimensie van X_test_poly:  (1345, 528)
R2 score op test set via L2:  0.5448161356757117
R2 score op training set via L2:  0.7195953061361415


c:\users\matth\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.67994e-26): result may not be accurate.
  overwrite_a=True).T


R2 score op test set via L1:  0.5649426086531691
R2 score op training set via L1:  0.5998010210919422


c:\users\matth\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 336.0189561714432, tolerance: 0.16419181082589263
  positive)


In [ ]:
# Voorspelling van de waardes.
values = np.array([7.3,0.70,0,1.9,0.079,11.0,34.0,0.9978,3.51,0.58,9.4])
score = lregmodel.predict(values.reshape(1,-1))
print("De score van de wijn is: " + str(score))

In [28]:
X.head()

,ID,Sex,Measurement_Age_x,Add_x,Sph-Far-R_x,Cyl-Far-R_x,Axis-Far-R_x,Sph-Close-R_x,Cyl-Close-R_x,Axis-Close-R_x,Sph-Far-L_x,Cyl-Far-L_x,Axis-Far-L_x,Sph-Close-L_x,Cyl-Close-L_x,Axis-Close-L_x,Measurement_Age_y,Add_y,Sph-Far-R_y,Cyl-Far-R_y,Axis-Far-R_y,Sph-Close-R_y,Cyl-Close-R_y,Axis-Close-R_y,Sph-Far-L_y,Cyl-Far-L_y,Axis-Far-L_y,Sph-Close-L_y,Cyl-Close-L_y,Axis-Close-L_y,Measurement_Age
81273,100104.0,1.0,21234.0,0.0,-5.00,2.50,80.0,0.0,0.0,0.0,-2.25,1.75,105.0,0.0,0.00,0.0,21234.0,0.00,-5.00,2.50,80.0,0.00,0.0,0.0,-2.25,1.75,105.0,0.00,0.00,0.0,21234.0
3774,100227.0,0.0,15496.0,0.0,0.00,0.00,0.0,1.0,0.0,0.0,0.00,0.00,0.0,1.0,0.00,0.0,16206.0,0.00,0.00,0.00,0.0,1.50,0.0,0.0,0.00,0.00,0.0,1.50,0.00,0.0,16799.0
166179,100337.0,1.0,9543.0,0.0,-6.50,0.00,0.0,0.0,0.0,0.0,-4.50,0.00,0.0,0.0,0.00,0.0,9696.0,0.00,-6.50,0.00,0.0,0.00,0.0,0.0,-4.50,0.00,0.0,0.00,0.00,0.0,12046.0
11739,100592.0,0.0,3011.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,-0.75,0.75,180.0,0.0,0.00,0.0,3935.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,-1.00,1.00,180.0,0.00,0.00,0.0,13217.0
22701,101121.0,1.0,15008.0,1.0,-2.75,0.75,10.0,-1.5,0.5,10.0,-2.25,0.50,160.0,-1.5,0.75,160.0,17566.0,1.25,-2.75,0.75,10.0,-1.25,0.5,10.0,-2.25,0.50,160.0,-1.25,0.75,160.0,18537.0


In [10]:
print(dataset.shape)
dataset.head(50)

(6721, 44)


,ID,Sex,Measurement_Age_x,Add_x,Sph-Far-R_x,Cyl-Far-R_x,Axis-Far-R_x,Sph-Close-R_x,Cyl-Close-R_x,Axis-Close-R_x,Sph-Far-L_x,Cyl-Far-L_x,Axis-Far-L_x,Sph-Close-L_x,Cyl-Close-L_x,Axis-Close-L_x,Measurement_Age_y,Add_y,Sph-Far-R_y,Cyl-Far-R_y,Axis-Far-R_y,Sph-Close-R_y,Cyl-Close-R_y,Axis-Close-R_y,Sph-Far-L_y,Cyl-Far-L_y,Axis-Far-L_y,Sph-Close-L_y,Cyl-Close-L_y,Axis-Close-L_y,Measurement_Age,Add,Sph-Far-R,Cyl-Far-R,Axis-Far-R,Sph-Close-R,Cyl-Close-R,Axis-Close-R,Sph-Far-L,Cyl-Far-L,Axis-Far-L,Sph-Close-L,Cyl-Close-L,Axis-Close-L
81273,100104.0,1.0,21234.0,0.00,-5.00,2.50,80.0,0.00,0.00,0.0,-2.25,1.75,105.0,0.00,0.00,0.0,21234.0,0.00,-5.00,2.50,80.0,0.00,0.00,0.0,-2.25,1.75,105.0,0.00,0.00,0.0,21234.0,0.00,-5.00,2.50,80.0,0.00,0.00,0.0,-2.25,1.75,105.0,0.00,0.00,0.0
3774,100227.0,0.0,15496.0,0.00,0.00,0.00,0.0,1.00,0.00,0.0,0.00,0.00,0.0,1.00,0.00,0.0,16206.0,0.00,0.00,0.00,0.0,1.50,0.00,0.0,0.00,0.00,0.0,1.50,0.00,0.0,16799.0,0.00,1.50,0.00,0.0,1.50,0.00,0.0,1.50,0.00,0.0,1.50,0.00,0.0
166179,100337.0,1.0,9543.0,0.00,-6.50,0.00,0.0,0.00,0.00,0.0,-4.50,0.00,0.0,0.00,0.00,0.0,9696.0,0.00,-6.50,0.00,0.0,0.00,0.00,0.0,-4.50,0.00,0.0,0.00,0.00,0.0,12046.0,0.00,-6.50,0.00,0.0,-6.50,0.00,0.0,-4.50,0.00,0.0,-4.50,0.00,0.0
11739,100592.0,0.0,3011.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,-0.75,0.75,180.0,0.00,0.00,0.0,3935.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,-1.00,1.00,180.0,0.00,0.00,0.0,13217.0,0.00,-2.50,1.75,10.0,-2.50,1.75,10.0,-2.00,1.25,180.0,-2.00,1.25,180.0
22701,101121.0,1.0,15008.0,1.00,-2.75,0.75,10.0,-1.50,0.50,10.0,-2.25,0.50,160.0,-1.50,0.75,160.0,17566.0,1.25,-2.75,0.75,10.0,-1.25,0.50,10.0,-2.25,0.50,160.0,-1.25,0.75,160.0,18537.0,2.25,-2.75,0.75,175.0,-1.00,1.25,175.0,-3.25,1.25,165.0,-0.50,0.75,165.0
86340,101146.0,1.0,19260.0,0.00,0.00,0.00,0.0,2.75,0.00,0.0,0.00,0.00,0.0,2.50,0.00,0.0,20355.0,0.00,0.00,0.00,0.0,3.00,0.00,0.0,0.00,0.00,0.0,2.75,0.00,0.0,20355.0,0.00,3.00,0.00,0.0,3.00,0.00,0.0,2.75,0.00,0.0,2.75,0.00,0.0
149844,101211.0,1.0,19954.0,2.00,1.25,0.50,180.0,3.25,0.50,180.0,1.25,0.50,180.0,3.25,0.50,180.0,19959.0,2.00,1.25,0.50,180.0,3.25,0.50,180.0,1.25,0.50,180.0,3.25,0.50,180.0,22160.0,2.50,1.50,0.75,175.0,4.00,0.75,175.0,1.50,0.75,10.0,4.00,0.75,10.0
129594,101287.0,1.0,18579.0,0.00,0.75,0.50,0.0,0.00,0.00,0.0,0.50,0.50,0.0,0.00,0.00,0.0,19635.0,0.00,0.00,0.00,0.0,3.25,0.75,180.0,0.00,0.00,0.0,2.75,0.75,180.0,19636.0,0.00,1.00,0.75,180.0,1.00,0.75,180.0,0.50,0.75,180.0,0.50,0.75,180.0
100146,101430.0,1.0,17110.0,0.00,-1.00,1.00,90.0,-1.00,1.00,90.0,-1.00,0.50,90.0,-1.00,0.50,90.0,17110.0,0.00,0.00,0.00,0.0,1.00,0.50,90.0,0.00,0.00,0.0,0.00,1.00,90.0,17110.0,0.00,-1.00,1.00,90.0,-1.00,1.00,90.0,-1.00,0.50,90.0,-1.00,0.50,90.0
39657,102096.0,0.0,3063.0,0.00,6.00,0.00,0.0,0.00,0.00,0.0,7.50,0.00,0.0,0.00,0.00,0.0,3994.0,0.00,5.00,0.00,0.0,0.00,0.00,0.0,6.50,0.00,0.0,0.00,0.00,0.0,4988.0,0.00,4.00,0.00,0.0,4.00,0.00,0.0,5.50,0.00,0.0,5.50,0.00,0.0
